# A Toy Trading System [Test]

I wanted to work on the node class, but that's hard without having an application/demo that is meaty enough to
work with.

So, here's a toy front-office position management, pricing, and risk system:

In [22]:
import mand.core

from mand.core import Entity, node, Context

from mand.core import ObjectDb, _tr, Timestamp, Context
from mand.core import ProfileMonitor
from mand.lib.extrefdata import ExternalRefData, dataField
from mand.lib.workflow import Workbook, WorkItemOpenEvent, WorkItem
from mand.lib.portfolio import Portfolio
from mand.core import displayDict, displayMarkdown
from mand.core import num, find
from mand.lib.dbsetup import setUpDb
import datetime

db = ObjectDb()
setUpDb(db)

## Books and Portfolios [BA]

These are just workflow entities that support a net present value function. As long as items in books are valuable
(in the sense of implementing NPV) then everything just works.

In [23]:
class TradingBook(Workbook):
    @node
    def NPV(self):
        ret = 0
        for i, q in self.items().items():
            ret += i.NPV() * q
        return ret

class TradingPortfolio(Portfolio):
    @node
    def NPV(self):
        ret = 0
        for i, q in self.items().items():
            ret += i.NPV() * q
        return ret
    
_tr.add(TradingBook)
_tr.add(TradingPortfolio)

## Building a tree of bank-side accounts [User]

Here, we build a tree of 100 books, grouped under 10 sub-portfolios, rooted in one top-level portfolio.

In a real bank, we might have hundreds of thousands of books, grouped 10 levels deep. in multiple different trees. 
Plus millions of customer books grouped in various portfolio trees.

In [24]:
def makeTree(names):
    ret = []
    for name in names:
        subs = [ TradingBook(name+str(i)) for i in range(10) ]
        p = TradingPortfolio(name).write()
        p.setChildren(subs)
        ret.append(p)
    return ret

with db:
    pAll = TradingPortfolio('TopOfTheHouse').write()
    subs = makeTree(['FX', 'Rates', 'Credit', 'Eq-Prop', 'Delta1', 'Eq-Inst', 'Loans', 'Commod', 'ETFs', 'Mtge'])
    pAll.setChildren(subs)
    
print pAll
print '# books:', len(pAll.books())
print '# children:', len(pAll.children())

# books: 100
# children: 10


## Checking execution counts [Test]

We have 11 portfolios, 100 books. We should get node calculation counts that reflect this...

In [25]:
with ProfileMonitor(mode='sum'):
    db2 = db.copy()
    p = db2._get(pAll.meta.path())
    print len(p.books())

100



### Profile by nodes.
* times are in microseconds
* cumT is total time spent in funtion
* calcT is time spent in function, but not in a child node

|fn|n|cumT|calcT|cumT/call|sys|
|-|-|-|-|-|-|
|Portfolio:books|11|1,244,983|292|113,180|GetValue
|Portfolio:books|11|1,244,956|1,245|113,177|GetValue/Calc
|Portfolio:children|11|1,243,446|306|113,040|GetValue
|Portfolio:children|11|1,243,139|106,406|113,012|GetValue/Calc
|PortfolioUpdateEvent:children|11|989,123|2,768|89,920|GetValue
|TradingBook|100|906,201|906,201|9,062|Db.Get
|PortfolioUpdateEvent|11|119,893|119,893|10,899|Db.Get
|TradingPortfolio|10|80,153|80,153|8,015|Db.Get
|Clock:cutoffs|22|18,540|204|842|GetValue
|Clock:cutoffs|1|18,336|46|18,336|GetValue/Calc
|Clock:parent|1|18,282|18|18,282|GetValue
|Clock:parent|1|18,264|8,533|18,264|GetValue/Calc
|CosmicAll|1|12,057|12,057|12,057|Db.Get
|Entity:clock|2|9,589|48|4,794|GetValue
|Entity:clock|1|9,541|51|9,541|GetValue/Calc
|RootClock|1|9,490|9,490|9,490|Db.Get
|Portfolio:clock|22|8,983|464|408|GetValue
|Portfolio:clock|11|8,518|352|774|GetValue/Calc
|Clock|1|8,166|8,166|8,166|Db.Get
|Event:amends|11|192|192|17|GetValue
|RootClock:cutoffs|3|148|31|49|GetValue
|RootClock:cutoffs|1|116|55|116|GetValue/Calc
|RootClock:cosmicAll|1|35|20|35|GetValue
|CosmicAll:dbState|1|25|15|25|GetValue
|RootClock:cosmicAll|1|15|15|15|GetValue/Calc
|CosmicAll:dbState|1|10|10|10|GetValue/Calc

## A few books [Test]

We also create two customers' books.

Pretend *p1* is one of our trading desks, *p2* is another. 

In [26]:
with db:
    bExt  = _tr.TradingBook('Customer1')
    bExt2 = _tr.TradingBook('Customer2')
    
p1 = pAll.children()[3]
p2 = pAll.children()[5]

b1 = p1.children()[4]
b2 = p2.children()[7]

print bExt.meta.name()
print b1.meta.name()
print b2.meta.name()

Customer1
Eq-Prop4
Eq-Inst7


## Tracking Market Data [DBA]

For now, just save incoming market observations as reference data. 

In reality, we'd have billions of observations per day on millions of data sources.

In [27]:
class MarketDataSource(ExternalRefData):
    @dataField
    def last(self):
        return None
    
_tr.add(MarketDataSource)

## Making some data sources [Test]

Two pretend data sources: IBM and Google last trade prices.

In [28]:
with db:
    s1_ibm  = MarketDataSource('source1.IBM')
    s1_goog = MarketDataSource('source1.GOOG')

s1_ibm.update(last=175.61)
s1_goog.update(last=852.12)

print s1_ibm.last()
print s1_goog.last()

175.61
852.12


## Market Interface [DBA]

How user code accesses market data. 

In reality, market interfaces would be choosing amongst raw data sources,
providing bootstrapped curves, vol surfaces, etc.

In [29]:
class MarketInterface(Entity):
    
    @node
    def sourceName(self):
        return 'source1'
    
    @node
    def source(self):
        return self.getObj(_tr.MarketDataSource, '%s.%s' % (self.sourceName(), self.meta.name()))
    
    @node
    def spot(self):
        return self.source().last()
                           
    
_tr.add(MarketInterface)


## Making some market interfaces [Test]

In [30]:
with db:
    ibm  = MarketInterface('IBM')
    goog = MarketInterface('GOOG')

print ibm.spot(), type(ibm.spot())
print goog.spot(), type(goog.spot())

175.61 <class 'decimal.Decimal'>
852.12 <class 'decimal.Decimal'>


## Instruments [BA]

Things we can own/have contracted. Instruments have an NPV.

We implement two classes of instrument:
* ForwardCashflow: money that will arrive at some future time
* Equity: a share of common stock in a company
    
Both implementations are toys.

In [31]:
class Instrument(WorkItem):
    """A thing that can be owned, an asset, or legal obligation"""
    
class ForwardCashflow(Instrument):
    
    @node(stored=True)
    def currency(self):
        return 'USD'
    
    @node(stored=True)
    def settlementDatetime(self):
        d = datetime.datetime.utcnow() + datetime.timedelta(2)
        return d
    
    @node
    def NPV(self):
        # XXX - would really get the currency discount curve here, and discount according to 
        # current time/settlement time
        # XXX - and do a conversion to our native currency
        return 1
    
    @node
    def name(self):
        return 'Cash %s/%s' % (self.currency(), self.settlementDatetime())
    
class Equity(Instrument):
    
    @node(stored=True)
    def assetName(self):
        return 'IBM.Eq.1'
    
    @node
    def NPV(self):
        return self.refdata().spot()
    
    @node
    def refdata(self):
        return self.getObj(_tr.MarketInterface, self.assetName().split('.')[0])

    @node
    def name(self):
        return 'Stock: %s' % self.assetName()
    
_tr.add(ForwardCashflow)
_tr.add(Equity)

## TradeOpenEvent [BA]

An event/observation that represents buying or selling something.

Someone gets *quantity* of *item*, and pays *unitPrice* of *premium* for each *item*.

Note that *premium* is typically a *ForwardCashflow*, so the money settles T+2. From a risk/PnL point
of view, we get the asset immediately. This is a gross over-simplification.

In [32]:
class TradeOpenEvent(WorkItemOpenEvent):
    @node(stored=True)
    def action(self):
        return 'Buy'
    @node(stored=True)
    def quantity(self):
        return 1.
    @node(stored=True)
    def premium(self):
        return None
    @node(stored=True)
    def unitPrice(self):
        return 0.
    
    def _items(self):
        bs = 1 if self.action() == 'Buy' else -1
        pq = -bs * self.unitPrice() * self.quantity()
        return [ [ self.ticket(), self.item(),    bs*self.quantity(), self.book1(), self.book2() ],
                 [ self.ticket(), self.premium(), pq,                 self.book1(), self.book2() ]
               ] 

_tr.add(TradeOpenEvent)

## Let's book some trades [Test]

In [33]:
with db:
    cf1 = ForwardCashflow()
    ins1 = Equity()
    ins2 = Equity(assetName='GOOG.Eq.1')
    
    ts1 = Timestamp()
    
    ev1 = TradeOpenEvent(action='Buy',
                         item=ins1,
                         quantity=100,
                         premium=cf1,
                         unitPrice=175.65,
                         book1=b1,
                         book2=bExt).write()
    
    ts2 = Timestamp()
    
    s1_ibm.update(last=175.64)
    
    ts3 = Timestamp()
    
    ev2 = TradeOpenEvent(action='Buy',
                         item=ins2,
                         quantity=300,
                         premium=cf1,
                         unitPrice=852.12,
                         book1=b2,
                         book2=bExt).write()
    
    ev3 = TradeOpenEvent(action='Sell',
                         item=ins1,
                         quantity=100,
                         premium=cf1,
                         unitPrice=175.85,
                         book1=b2,
                         book2=bExt2).write()
    
    ts4 = Timestamp()
    
    s1_ibm.update(last=175.70)
    s1_goog.update(last=852.11)
    
    ts5 = Timestamp()
    
    s1_ibm.update(last=175.68)
    s1_goog.update(last=852.13)
    
    eod = Timestamp()
    
    ev4 = TradeOpenEvent(action='Buy',
                         item=ins1,
                         quantity=100,
                         premium=cf1,
                         unitPrice=175.69,
                         book1=b1,
                         book2=bExt,
                         amends=ev1,
                         message='Sorry, the broker says you actually paid 69. signed: the middle office'
                        ).write(validTime=ev1.meta._timestamp.validTime)
    
    s1_ibm.update(last=177.68)
    s1_goog.update(last=856.13)
    
    ts6 = Timestamp()
    

# Reporting [User]

Reporting is the whole point of the Abstract Nonsense Db.

We ignore all the standard report infrastructure goop (aggregation, GUIs, pivot-tables, drill-down, etc) and
just focus on extracting the underlying data.

For the following examples, we mostly care about NPV (the present values of what we own,) 
PnL (how much money we made or lost,) and risk (what happens to our NPV if underlying conditions change.)

Note that even in the trivial case of our three trades, one amendment, and several market data updates, trying to write
code to attribute profit, or even trying to figure it by hand, would require assumptions and be error-prone. 

In general, the less a report knows about the underlying data, the better it is.

## Report: PnL [User]

How much money did we (i.e. our portfolio) gain or lose between two times?

For example, if time1 was yesterday's close, and time2 is today's close, this report tells us how much we made or lost
today. Will we be drinking champagne or straight vodka this evening?

Note: it's worth understanding how the cash entries are working in this example. The reason our open and closed out
positions are pricing rationally is because we are modelling cash approximately correctly, not because the instruments
are carrying around a notion of price-paid!

In [34]:
def reportPnL(valuable, ts1, ts2):
    clock = valuable.getObj(_tr.RootClock, 'Main')
    with Context({clock.cutoffs: ts1}):
        npv1 = valuable.NPV()
    with Context({clock.cutoffs: ts2}):
        npv2 = valuable.NPV()
    displayMarkdown('#### PnL for %s: %s' % (valuable.meta.name(), npv2-npv1))

### Example: top-of-the-house intraday PnL

Hmm, one of our desks bought 100 shares of IBM, and the last trade price is already down 1 cent. The bank is down 
100 times $.01 or one dollar:

In [35]:
# 100 shares bought at $175.65
# current price is $175.64:
reportPnL(pAll, ts1, ts3)

#### PnL for TopOfTheHouse: -1.00

### Example: top-of-the-house daily PnL

By end of day, the bank as a whole has closed out the IBM trade for a profit, and bought some GOOG. Closing refdata
prices have been entered. We should have a total profit for the day of $23:

In [36]:
# 100 * ($175.85 - $175.65) =  $20
# 300 * ($852.13 - $852.12) =  $ 3
#                              ----
#                              $23
reportPnL(pAll, ts1, eod)

#### PnL for TopOfTheHouse: 23.00

### Example: trading desk daily PnL

Desk *p1* bought IBM at $175.65

It still owns IBM, which closed at $175.68

So, it should be up $3:

In [37]:
# 100 * ($175.68 - $175.65) =   $3
reportPnL(p1, ts1, eod)

#### PnL for Eq-Prop: 3.00

## Report: Market Risk [DBA]

So, how much does our value change if market data values change?

Note: review the greeks workbook if this is unclear.

In [38]:
def reportMarketRisk(valuable):
    def fn(node):
        obj = node.key[0]
        m = node.key[1].split(':')[-1]
        if isinstance(obj, MarketInterface) and m == 'spot':
            return True
    nodes = find(valuable.NPV, fn)
    r = {}
    npv = valuable.NPV()
    for n in nodes:
        v = n.value * num(1.01)
        key = n.tweakPoint
        with Context({key: v}):
            obj = n.key[0]
            npv2 = valuable.NPV()
            r[obj.meta.name()] = npv2-npv
    displayMarkdown('### %s: NPV change under a 1%% move' % valuable.meta.name())
    displayDict(r)

In [39]:
reportMarketRisk(pAll)
reportMarketRisk(p1)
reportMarketRisk(p2)

### TopOfTheHouse: NPV change under a 1% move

|key|value|
|-|-|
|GOOG|2568.3900

### Eq-Prop: NPV change under a 1% move

|key|value|
|-|-|
|IBM|177.6800

### Eq-Inst: NPV change under a 1% move

|key|value|
|-|-|
|GOOG|2568.3900
|IBM|-177.6800

## Report: Prior Day Amends [User]

So, someone reported a number on day T. How would that number change if we reran the report based on corrected data?

There are a lot of ways to handle prior day amends:
* Have them entered, but have no systemic way to track them.
 * This is a time-honoured approach. Usually, someone goes to jail when the auditors eventually notice that some trader
declaring profits each day is just amending the trades the next day and actually losing money.
* Make a policy that there are no prior day amends.
 * Well, now at least you can see the PnL.
 * On the downside, you don't get much explanatory power. It's a bit like switching to cash accounting because your
internal business units are gaming accrual accounting to the point you can't even manage the budget.
 * Oh, and you still get fined/go to jail: if the number is your position size, and you exceeded the legal limit on
day T, you can't just pretend you didn't and book your day T over-limit trade on day T+1. External reality does not
change just because you have a bad operational policy.
* Just do it right.
 * This is the approach we take.

In [40]:
def reportRestates(valuable, ts1, ts2):
    clock = valuable.getObj(_tr.RootClock, 'Main')
    with Context({clock.cutoffs: ts1}):
        npv1 = valuable.NPV()
    print
    ts1r = Timestamp(t=ts2.transactionTime, v=ts1.validTime)
    with Context({clock.cutoffs: ts1r}):
        npv2 = valuable.NPV()
    if npv2-npv1:
        displayMarkdown('#### AUDIT-ITEM. %s: NPV change due to prior day amends is %s' % 
                        (valuable.meta.name(), npv2-npv1))

In [41]:
reportRestates(p1, eod, ts6)

print
print 'Note that we are only looking at amended numbers, not all activity:'
reportPnL(p1, eod, ts6)

#### AUDIT-ITEM. Eq-Prop: NPV change due to prior day amends is -4.00


Note that we are only looking at amended numbers, not all activity:


#### PnL for Eq-Prop: 196.00

# Let's see what is going on when we run a report [Core]

In [42]:
with ProfileMonitor(mode='sum'):
    db3 = db.copy()
    p = db3._get(pAll.meta.path())
    reportPnL(p, ts1, ts6)

#### PnL for TopOfTheHouse: 1219.00


### Profile by nodes.
* times are in microseconds
* cumT is total time spent in funtion
* calcT is time spent in function, but not in a child node

|fn|n|cumT|calcT|cumT/call|sys|
|-|-|-|-|-|-|
|Root:4427354008|2|6,900,897|151|3,450,448|Context
|TradingPortfolio:NPV|2|6,900,746|83|3,450,373|GetValue
|TradingPortfolio:NPV|2|6,900,662|599|3,450,331|GetValue/Calc
|Portfolio:items|22|6,688,719|787|304,032|GetValue
|Portfolio:items|22|6,688,658|12,584|304,029|GetValue/Calc
|Workbook:items|200|3,910,445|6,786|19,552|GetValue
|Workbook:items|200|3,903,659|3,590,821|19,518|GetValue/Calc
|Portfolio:children|44|2,761,345|1,032|62,757|GetValue
|Portfolio:children|22|2,760,312|417,002|125,468|GetValue/Calc
|PortfolioUpdateEvent:children|22|2,027,905|4,240|92,177|GetValue
|TradingBook|102|1,875,801|1,875,801|18,390|Db.Get
|PortfolioUpdateEvent|11|234,638|234,638|21,330|Db.Get
|Equity:NPV|1|211,294|45|211,294|GetValue
|Equity:NPV|1|211,249|128|211,249|GetValue/Calc
|MarketInterface:spot|1|188,496|47|188,496|GetValue
|MarketInterface:spot|1|188,449|148|188,449|GetValue/Calc
|TradingPortfolio|10|178,043|178,043|17,804|Db.Get
|ExternalRefData:state|1|165,826|46|165,826|GetValue
|ExternalRefData:state|1|165,780|109|165,780|GetValue/Calc
|RefData:state|1|165,670|54|165,670|GetValue
|RefData:state|1|165,616|16,169|165,616|GetValue/Calc
|RefDataUpdateEvent|4|111,268|111,268|27,817|Db.Get
|Clock:cutoffs|446|98,881|5,470|221|GetValue
|Clock:cutoffs|5|93,410|452|18,682|GetValue/Calc
|Clock:parent|5|92,896|155|18,579|GetValue
|Clock:parent|5|92,740|92,007|18,548|GetValue/Calc
|TradeOpenEvent|4|85,400|85,400|21,350|Db.Get
|Clock|3|66,801|66,801|22,267|Db.Get
|Workbook:clock|400|55,546|13,438|138|GetValue
|_WorkItemEvent:ticket|6|42,828|203|7,138|GetValue
|WorkTicket|3|42,624|42,624|14,208|Db.Get
|Workbook:clock|200|42,107|20,120|210|GetValue/Calc
|_WorkItemEvent:item|3|39,792|155|13,264|GetValue
|Equity|2|39,636|39,636|19,818|Db.Get
|_WorkItemEvent:book2|6|30,337|158|5,056|GetValue
|CosmicAll|1|25,971|25,971|25,971|Db.Get
|Portfolio:clock|44|24,651|1,397|560|GetValue
|RootClock|1|23,319|23,319|23,319|Db.Get
|Portfolio:clock|22|23,254|1,039|1,057|GetValue/Calc
|RefData:clock|2|22,742|61|11,371|GetValue
|RefData:clock|1|22,680|80|22,680|GetValue/Calc
|Equity:refdata|1|22,624|40|22,624|GetValue
|Equity:refdata|1|22,583|142|22,583|GetValue/Calc
|MarketInterface:source|1|22,475|36|22,475|GetValue
|MarketInterface:source|1|22,439|138|22,439|GetValue/Calc
|MarketInterface|1|22,416|22,416|22,416|Db.Get
|MarketDataSource|1|22,253|22,253|22,253|Db.Get
|TradeOpenEvent:premium|3|20,189|111|6,729|GetValue
|ForwardCashflow|1|20,077|20,077|20,077|Db.Get
|Event:amends|29|10,937|10,937|377|GetValue
|Portfolio:books|42|3,799|1,003|90|GetValue
|Portfolio:books|22|3,044|2,552|138|GetValue/Calc
|Entity:clock|10|598|395|59|GetValue
|Entity:clock|5|202|202|40|GetValue/Calc
|RootClock:cutoffs|15|196|196|13|GetValue
|RefDataUpdateEvent:data|4|140|140|35|GetValue
|_WorkItemEvent:book1|6|123|123|20|GetValue
|TradeOpenEvent:quantity|6|98|98|16|GetValue
|TradeOpenEvent:action|3|59|59|19|GetValue
|TradeOpenEvent:unitPrice|3|53|53|17|GetValue
|ForwardCashflow:NPV|1|49|36|49|GetValue
|MarketInterface:sourceName|1|46|30|46|GetValue
|Equity:assetName|1|25|25|25|GetValue
|MarketInterface:sourceName|1|15|15|15|GetValue/Calc
|ForwardCashflow:NPV|1|12|12|12|GetValue/Calc